In [1]:
# Importing necessary libraries
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance

# Defining the provided functions and parameters

# Define the weight function
def omega(t, u, x, c1, c2):
    sigma = math.sqrt(np.var(Y))
    h1 = (c1/math.sqrt(12))*(T ** (-(1/5)))
    h2 = c2*sigma*(T ** (-(1/5)))
    numerator = K_h1(u - t / T, h1) * np.prod(K_h2(x - Y[t-1], h2))
    denominator = np.sum([K_h1(u - s / T, h1) * np.prod(K_h2(x - Y[s-1], h2)) for s in range(1, T+1)])
    return numerator / denominator

def ecdf(u, x, v):
    weights = np.array([omega(t, u, x, c1, c2) for t in range(1, T+1)])
    indicator = (Y <= v).astype(int)
    return np.sum(weights * indicator)

# Define uniform kernels
def K_h1(z, h1):
    return 0.5 if abs(z/h1) <= 1 else 0

# Define Gaussian kernels
def K_h2(z, h2):
    return np.exp(-z**2 / (2 * h2**2))

# Parameters
T = 200  # Length of time series
d = 1  # Number of covariates
c1 = 5
c2 = 1.68
u_test = 0.5
x_test = np.array([0.5])
v_values = np.linspace(-2, 2, T)  # Placeholder values, will be updated in the loop

# Running the replication loop
W1_list = []

for rep in range(100):
    # Generate locally stationary data
    np.random.seed(rep)  # Use different seed for each replication
    Y = np.sin(np.linspace(0, 4 * np.pi, T)) + np.random.normal(0, 0.5, T)  # Time series with noise
    
    v_values = np.linspace(Y.min(), Y.max(), T)  # Update v_values based on new Y
    
    h1 = (c1/math.sqrt(12))*(T ** (-(1/5)))
    h2 = c2*math.sqrt(np.var(Y))*(T ** (-(1/5)))
    
    # Test the weight function for a given (u, x)
    weights = [omega(t, u_test, x_test, c1, c2) for t in range(1, T+1)]
    
    # Values for plotting
    F_values = [ecdf(u_test, x_test, v) for v in v_values]
    
    # True CDF
    data_sorted = np.sort(Y)
    True_CDF = np.arange(1, len(data_sorted) + 1) / len(data_sorted)
    
    # Compute the 1-Wasserstein distance
    W1 = wasserstein_distance(True_CDF, F_values)
    W1_list.append(W1)

# Computing mean 1-Wasserstein distance
mean_W1 = np.mean(W1_list)

print("1-Wasserstein Distance: ", W1_list)
print("Average 1-Wasserstein Distance: ", mean_W1)


1-Wasserstein Distance:  [0.16559458114649378, 0.1890488686933917, 0.1897108544123151, 0.1982160796072429, 0.1797220766426411, 0.19707218850266756, 0.18249815119776663, 0.1668923261644728, 0.18376607498958902, 0.1667121184285662, 0.1785862929136241, 0.17702163496222634, 0.16685443768559124, 0.17374383178939248, 0.1802283855195065, 0.19302910730704406, 0.17132096738713076, 0.19059405150582184, 0.1707357347505587, 0.213006262090907, 0.1883859261978027, 0.21716035444949047, 0.168019059729992, 0.1752328323417574, 0.18801695415534353, 0.18511252905588657, 0.1741166907333793, 0.21030288103007277, 0.2256913323345074, 0.20558971639129187, 0.18354898716774398, 0.19792834263736808, 0.16969388955064882, 0.19304528525232223, 0.18287983446223538, 0.18613303834858042, 0.16348000818176622, 0.18899032187754158, 0.16871238790325394, 0.1689844711531976, 0.18037432594159952, 0.19328730664669738, 0.19460803574619034, 0.1656949867571602, 0.17547987090241823, 0.17638401474382637, 0.18019308112414906, 0.1838